In [222]:
%matplotlib ipympl

In [2]:
from h3 import h3

In [1]:
import sys
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.ops import transform
import pyproj
import math
sys.path.append('..')
import matplotlib.pyplot as plt
from dggrid4py import DGGRIDv7, Dggs, dgselect, dggs_types

In [5]:
dir = os.getcwd()

In [6]:
dir

'/mnt/d/UTCloud/DGGS/working/Python/PyProj/notebooks'

In [2]:
dggrid = DGGRIDv7(executable='/mnt/d/UTCloud/DGGS/working/Python/PyProj/cell_stats/dggrid', working_dir='/mnt/d/UTCloud/DGGS/working/Python/PyProj/cell_stats/dggrid_workdir', capture_logs=False, silent=False)

In [5]:
gdf = dggrid.grid_cell_polygons_for_extent('FULLER4D', 7)

** executing DGGRID version 7.07 **
type sizes: big int: 64 bits / big double: 128 bits

** loading meta file metafile_3cbcfaa8-a8a8-414b-83fa-18c4f12287ed...
* using parameter values:
dggrid_operation GENERATE_GRID (user set)
dggs_type FULLER4D (user set)
dggs_topology DIAMOND (user set)
dggs_proj FULLER (user set)
dggs_aperture_type PURE (user set)
dggs_aperture 4 (user set)
proj_datum WGS84_AUTHALIC_SPHERE (default)
dggs_orient_specify_type SPECIFIED (user set)
dggs_num_placements 1 (user set)
dggs_orient_output_file_name grid.meta (default)
dggs_vert0_lon 11.25 (user set)
dggs_vert0_lat 58.2825 (user set)
dggs_vert0_azimuth 0 (user set)
dggs_res_specify_type SPECIFIED (user set)
dggs_res_spec 7 (user set)
rng_type RAND (default)
geodetic_densify 0 (default)
clip_subset_type WHOLE_EARTH (user set)
clip_region_files test.gen (default)
clip_type POLY_INTERSECT (default)
clipper_scale_factor 1000000 (default)
densification 0 (default)
precision 7 (default)
output_cell_label_type GLOBAL

In [6]:
gdf.to_file('/mnt/d/UTCloud/DGGS/grids/grids.gpkg', layer='DGGRID_FULLER_4D', driver='GPKG')

In [30]:
# Area on sphere from lat lon
def degree_to_rad(degree):
    return degree * pi/180

def calculate_poly_area(geom):
    area = 0
    for i in range (0,len(geom.exterior.coords)-1):
        p1 = geom.exterior.coords[i]
        p2 = geom.exterior.coords[i+1]
        area = area + degree_to_rad(p2[0] - p1[0])*(2+sin(degree_to_rad(p1[1])) + sin(degree_to_rad(p2[1])))
    area = area * 6378137 * 6378137 / 2
    
    return abs(area)/1000000

In [181]:
def check_crossing(lon1: float, lon2: float, validate: bool = True):
    """
    Assuming a minimum travel distance between two provided longitude coordinates,
    checks if the 180th meridian (antimeridian) is crossed.
    """
    if validate and any(abs(x) > 180.0 for x in [lon1, lon2]):
        raise ValueError("longitudes must be in degrees [-180.0, 180.0]")   
    return abs(lon2 - lon1) > 180.0

def check_for_geom(geom):
    crossed = False
    p_init = geom.exterior.coords[0]

    for p in range(1, len(geom.exterior.coords)):
        px = geom.exterior.coords[p]
        # print(px)

        if check_crossing(p_init[0], px[0]):
            crossed = True
        p_init = px
    
    return crossed

In [179]:
grid = gpd.read_file(r'd:\UTCloud\DGGS\grids\grids.gpkg', layer='DGGRID_ISEA3H_9', driver='gpkg')

In [247]:
fig, ax = plt.subplots(figsize=(10,5))
grid[~grid['crossed']]['ind_lambert_area'].plot(kind='hist',bins=100,ax=ax)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [248]:
fig, ax = plt.subplots(figsize=(10,5))
grid['ind_lambert_area'].plot(kind='hist',bins=100,ax=ax)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [231]:
fig, ax = plt.subplots(figsize=(10,5))
pd.DataFrame({'raw':grid.std(),'filtered':grid[~grid['crossed']].std()}).transpose()[['ind_lambert_area','ind_UTM_area','isea_area','goode_area']].plot(kind='bar',ax=ax)
ax.set_title('ISEA3H 9 std')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [230]:
fig, ax = plt.subplots(figsize=(10,5))
pd.DataFrame({'raw':grid.std(),'filtered':grid[~grid['crossed']].std()}).transpose()[['ind_lambert_area','ind_UTM_area','goode_area']].plot(kind='bar',ax=ax)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [232]:
fig, ax = plt.subplots(figsize=(10,5))
pd.DataFrame({'filtered':grid[~grid['crossed']].std()}).transpose()[['ind_lambert_area','ind_UTM_area','goode_area']].plot(kind='bar',ax=ax)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [236]:
pd.DataFrame({'raw':grid.dropna().std(),'filtered':grid[~grid['crossed']].dropna().std()}).transpose()

,ind_lambert_area,ind_UTM_area,isea_area,goode_area,crossed
raw,2.991031e+10,3.984807e+09,7.427621e+10,6.099055e+10,0.049476
filtered,1.209756e+07,1.158927e+07,7.393763e+10,2.644275e+07,0.000000


In [205]:
grid[~grid['crossed']].std()

ind_lambert_area    1.209756e+07
ind_UTM_area        1.158927e+07
isea_area           7.393763e+10
goode_area          2.644275e+07
crossed             0.000000e+00
dtype: float64